In [1]:
!unzip /content/drive/MyDrive/crop-disease/train.zip # 자신의 train.zip 
!mkdir model

Archive:  /content/drive/MyDrive/crop-disease/train.zip
replace train/10027/10027.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
mkdir: cannot create directory ‘model’: File exists


In [2]:
!pip install timm

# 라이브러리 로드 및 필요 함수 정의

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import json
import cv2
import matplotlib.pyplot as plt

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import time



device = torch.device('cuda')

# score 계산
def accuracy_function(real, pred):                   # https://dacon.io/competitions/official/235870/codeshare/4146
    score = f1_score(real, pred, average='macro')
    return score

# torch model 저장
def model_save(model, score,  path):
    os.makedirs('model', exist_ok=True)
    torch.save({
        'model': model.state_dict(),
        'score': score
    }, path)



# 데이터 로드

In [2]:
train_png = sorted(glob('../data/train/*.png'))
test_png = sorted(glob('../data/test/*.png'))

train_y = pd.read_csv("../data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [3]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (256, 256))
    return img

In [4]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████████████████████████████████████████████████████████████████████████| 2154/2154 [00:43<00:00, 49.00it/s]


In [4]:
# # 주어진 훈련 데이터 경로
# train_csv = sorted(glob('train/*/*.csv'))
# train_jpg = sorted(glob('train/*/*.jpg'))
# train_json = sorted(glob('train/*/*.json'))


# crops = []
# diseases = []
# risks = []
# labels = []

# # 훈련데이터 로드
# for i in range(len(train_json)):
#     with open(train_json[i], 'r') as f:
#         sample = json.load(f)
#         crop = sample['annotations']['crop']
#         disease = sample['annotations']['disease']
#         risk = sample['annotations']['risk']
#         label=f"{crop}_{disease}_{risk}"
    
#         crops.append(crop)
#         diseases.append(disease)
#         risks.append(risk)
#         labels.append(label)

# label_unique = sorted(np.unique(labels))
# label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

# labels = [label_unique[k] for k in labels]


# # 이미지 로드 함수 / 로드 후 (512, 384)로 사이즈 변환 ==> (원본 사이즈에서 width 384인 샘플이 많아서 (512, 384) 사이즈로 변환)
# def img_load(path):
#     img = cv2.imread(path)[:,:,::-1]
#     img = cv2.resize(img, (384, 512))
#     return img

# # 이미지 로드
# imgs = [img_load(k) for k in tqdm(train_jpg)]

    

100%|██████████| 5767/5767 [00:38<00:00, 151.46it/s]


# 데이터로더와 모델 정의
- 모델에 입력되는 이미지에 다양성을 추가하기 위해 vertical, horizontal flip을 랜덤하게 적용. ( 훈련데이터만 )
- Efficientent-b0 (pretrained) 모델 사용.
- 실제로는 crop_disease_risk가 다양한 조합으로 나올 수 있으나 훈련데이터에 존재하는 label들의 고유값은 총 25개였기 때문에 출력값을 25로 설정. (test dataset에 존재하는 label은 train dataset에도 존재 https://dacon.io/competitions/official/235870/talkboard/405713?page=1&dtype=recent)

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode = mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            return img
        
        label = self.labels[idx]
        return img,label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=25)
        
    def forward(self, x):
        x = self.model(x)
        return x    

# 데이터 분리 및 로더 정의
- 검증 데이터는 간단히 KFold (k=5) 방식을 사용.
- 배치 사이즈는 16을 사용. colab(일반버전) 에서 batch size 32일 때 메모리 오류가 납니다.
- 에폭은 30으로 설정.

In [6]:
# KFold
folds = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, valid_idx in kf.split(train_imgs):
    folds.append((train_idx, valid_idx))
fold=0
train_idx, valid_idx = folds[fold]

batch_size = 16
epochs = 30


# Train
train_dataset = Custom_dataset(np.array(train_imgs)[train_idx], np.array(train_labels)[train_idx], mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=8)

# Validation 
valid_dataset = Custom_dataset(np.array(train_imgs)[valid_idx], np.array(train_labels)[valid_idx], mode='valid')
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=8)

# 훈련
- optimizer는 Adam을 사용.
- Mixed Precision Training을 사용하기위헤 AMP를 사용.
- validation score 를 모니터하면서 점수가 개선될 때 마다 모델을 저장.


In [ ]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

print("Running")

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        print("Running")
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = accuracy_function(train_y, train_pred)
    
    model.eval()
    valid_loss = 0
    valid_pred=[]
    valid_y=[]
    with torch.no_grad():
        for batch in (valid_loader):
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)
            valid_loss += loss.item()/len(valid_loader)
            valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            valid_y += y.detach().cpu().numpy().tolist()
        valid_f1 = accuracy_function(valid_y, valid_pred)
    if valid_f1>=best:
        best=valid_f1
        model_save(model, valid_f1, f'model/eff-b0.pth')
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'VALID    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}    best : {best:.5f}')

Running


In [6]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = accuracy_function(train_y, train_pred)
    
    model.eval()
    valid_loss = 0
    valid_pred=[]
    valid_y=[]
    with torch.no_grad():
        for batch in (valid_loader):
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)
            valid_loss += loss.item()/len(valid_loader)
            valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            valid_y += y.detach().cpu().numpy().tolist()
        valid_f1 = accuracy_function(valid_y, valid_pred)
    if valid_f1>=best:
        best=valid_f1
        model_save(model, valid_f1, f'model/eff-b0.pth')
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'VALID    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}    best : {best:.5f}')

epoch : 1/30    time : 110s/3178s
TRAIN    loss : 0.60060    f1 : 0.61207
VALID    loss : 0.35065    f1 : 0.67450    best : 0.67450
epoch : 2/30    time : 108s/3034s
TRAIN    loss : 0.23640    f1 : 0.81202
VALID    loss : 0.23117    f1 : 0.79577    best : 0.79577
epoch : 3/30    time : 108s/2922s
TRAIN    loss : 0.18012    f1 : 0.86794
VALID    loss : 0.26513    f1 : 0.82589    best : 0.82589
epoch : 4/30    time : 108s/2811s
TRAIN    loss : 0.17742    f1 : 0.86356
VALID    loss : 0.21942    f1 : 0.85821    best : 0.85821
epoch : 5/30    time : 108s/2703s
TRAIN    loss : 0.14122    f1 : 0.89331
VALID    loss : 0.23205    f1 : 0.87375    best : 0.87375
epoch : 6/30    time : 108s/2591s
TRAIN    loss : 0.11268    f1 : 0.92715
VALID    loss : 0.19384    f1 : 0.85435    best : 0.87375
epoch : 7/30    time : 108s/2479s
TRAIN    loss : 0.07634    f1 : 0.93744
VALID    loss : 0.16295    f1 : 0.88713    best : 0.88713
epoch : 8/30    time : 108s/2367s
TRAIN    loss : 0.08807    f1 : 0.94531
VA

In [7]:
!nvidia-smi

Thu Jan 13 08:38:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |   4908MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------